In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from sklearn import preprocessing, ensemble, metrics, linear_model, model_selection, inspection
import semopy
import datetime as dt
from scipy import interpolate
from tqdm import tqdm
from pprint import pprint

In [2]:
df_profile = pd.read_csv('../datasets/wosis_2019/wosis_201909_profiles.tsv', sep='\t')
profile_id_all_list = list(np.unique(df_profile['profile_id']))
print(profile_id_all_list[:10])
print(df_profile.shape)
print(list(df_profile.columns))
df_profile.head(2)

[36897, 36898, 36899, 36900, 36901, 36902, 36903, 36904, 36905, 36906]
(196498, 23)
['profile_id', 'dataset_id', 'country_id', 'country_name', 'geom_accuracy', 'latitude', 'longitude', 'dsds', 'cfao_version', 'cfao_major_group_code', 'cfao_major_group', 'cfao_soil_unit_code', 'cfao_soil_unit', 'cwrb_version', 'cwrb_reference_soil_group_code', 'cwrb_reference_soil_group', 'cwrb_prefix_qualifier', 'cwrb_suffix_qualifier', 'cstx_version', 'cstx_order_name', 'cstx_suborder', 'cstx_great_group', 'cstx_subgroup']


C:\Users\LZ\AppData\Local\Temp\ipykernel_72252\4109598479.py:1: DtypeWarning: Columns (9,10,11,12,14,15,16,17,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_profile = pd.read_csv('../datasets/wosis_2019/wosis_201909_profiles.tsv', sep='\t')


,profile_id,dataset_id,country_id,country_name,geom_accuracy,latitude,longitude,dsds,cfao_version,cfao_major_group_code,...,cwrb_version,cwrb_reference_soil_group_code,cwrb_reference_soil_group,cwrb_prefix_qualifier,cwrb_suffix_qualifier,cstx_version,cstx_order_name,cstx_suborder,cstx_great_group,cstx_subgroup
0,36897,{BE-UplandsI},BE,Belgium,0.000001,50.649889,4.666901,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36898,{BE-UplandsI},BE,Belgium,0.000001,50.583962,4.462114,97.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_physical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_physical.tsv', sep='\t')
print(df_physical.shape)
print(list(df_physical.columns))
df_physical.head(2)

C:\Users\LZ\AppData\Local\Temp\ipykernel_72252\2576909374.py:1: DtypeWarning: Columns (4,6,8,9,10,11,12,13,15,16,17,18,19,20,22,23,24,25,26,27,29,30,31,32,33,34,36,37,38,39,40,55,57,58,59,60,61,62,64,65,66,67,68,69,71,72,73,74,75,76,78,79,80,81,82,83,85,86,87,88,89,90,92,93,94,95,96,97,99,100,101,102,103,104,106,107,108,109,110,111,113,114,115,116,117,118,120,121,122,123,124,125,127,128,129,130,131,132,134,135,136,137,138,139,141,142,143,144,145,146,148,149,150,151,152,153,155,156,157,158,159,160,162,163,164,165,166,167,169,170,171,172,173,174,176,177,178,179,180,181,183,184,185,186,187,188,190,191,192,193,194) have mixed types. Specify dtype option on import or set low_memory=False.
  df_physical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_physical.tsv', sep='\t')


(702698, 195)
['profile_id', 'profile_layer_id', 'upper_depth', 'lower_depth', 'layer_name', 'litter', 'bdfi33_value', 'bdfi33_value_avg', 'bdfi33_method', 'bdfi33_date', 'bdfi33_dataset_id', 'bdfi33_profile_code', 'bdfi33_licence', 'bdfiad_value', 'bdfiad_value_avg', 'bdfiad_method', 'bdfiad_date', 'bdfiad_dataset_id', 'bdfiad_profile_code', 'bdfiad_licence', 'bdfifm_value', 'bdfifm_value_avg', 'bdfifm_method', 'bdfifm_date', 'bdfifm_dataset_id', 'bdfifm_profile_code', 'bdfifm_licence', 'bdfiod_value', 'bdfiod_value_avg', 'bdfiod_method', 'bdfiod_date', 'bdfiod_dataset_id', 'bdfiod_profile_code', 'bdfiod_licence', 'bdws33_value', 'bdws33_value_avg', 'bdws33_method', 'bdws33_date', 'bdws33_dataset_id', 'bdws33_profile_code', 'bdws33_licence', 'bdwsad_value', 'bdwsad_value_avg', 'bdwsad_method', 'bdwsad_date', 'bdwsad_dataset_id', 'bdwsad_profile_code', 'bdwsad_licence', 'bdwsfm_value', 'bdwsfm_value_avg', 'bdwsfm_method', 'bdwsfm_date', 'bdwsfm_dataset_id', 'bdwsfm_profile_code', 'bdws

,profile_id,profile_layer_id,upper_depth,lower_depth,layer_name,litter,bdfi33_value,bdfi33_value_avg,bdfi33_method,bdfi33_date,...,wv0500_dataset_id,wv0500_profile_code,wv0500_licence,wv0006_value,wv0006_value_avg,wv0006_method,wv0006_date,wv0006_dataset_id,wv0006_profile_code,wv0006_licence
0,47010,1,0.0,21.0,Ap,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47010,2,21.0,35.0,E1,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_chemical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_chemical.tsv', sep='\t')
print(df_chemical.shape)
print(list(df_chemical.columns))
df_chemical.head(2)

C:\Users\LZ\AppData\Local\Temp\ipykernel_72252\1985226948.py:1: DtypeWarning: Columns (4,6,8,9,10,11,12,13,15,16,17,18,19,20,22,23,24,25,26,27,29,30,31,32,33,34,36,37,38,39,40,41,43,44,45,46,47,48,50,51,52,53,54,55,57,58,59,60,61,62,64,65,66,67,68,69,71,72,73,74,75,76,78,79,80,81,82,83,85,86,87,88,89,90,92,93,94,95,96,97,99,100,101,102,103,104,106,107,108,109,110,111,113,114,115,116,117,118,120,121,122,123,124,125,127,128,129,130,131,132,134,135,136,137,138,139,141,142,143,144,145,146,148,149,150,151,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chemical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_chemical.tsv', sep='\t')


(788538, 153)
['profile_id', 'profile_layer_id', 'upper_depth', 'lower_depth', 'layer_name', 'litter', 'tceq_value', 'tceq_value_avg', 'tceq_method', 'tceq_date', 'tceq_dataset_id', 'tceq_profile_code', 'tceq_licence', 'cecph7_value', 'cecph7_value_avg', 'cecph7_method', 'cecph7_date', 'cecph7_dataset_id', 'cecph7_profile_code', 'cecph7_licence', 'cecph8_value', 'cecph8_value_avg', 'cecph8_method', 'cecph8_date', 'cecph8_dataset_id', 'cecph8_profile_code', 'cecph8_licence', 'ecec_value', 'ecec_value_avg', 'ecec_method', 'ecec_date', 'ecec_dataset_id', 'ecec_profile_code', 'ecec_licence', 'elco20_value', 'elco20_value_avg', 'elco20_method', 'elco20_date', 'elco20_dataset_id', 'elco20_profile_code', 'elco20_licence', 'elco25_value', 'elco25_value_avg', 'elco25_method', 'elco25_date', 'elco25_dataset_id', 'elco25_profile_code', 'elco25_licence', 'elco50_value', 'elco50_value_avg', 'elco50_method', 'elco50_date', 'elco50_dataset_id', 'elco50_profile_code', 'elco50_licence', 'elcosp_value',

,profile_id,profile_layer_id,upper_depth,lower_depth,layer_name,litter,tceq_value,tceq_value_avg,tceq_method,tceq_date,...,totc_dataset_id,totc_profile_code,totc_licence,nitkjd_value,nitkjd_value_avg,nitkjd_method,nitkjd_date,nitkjd_dataset_id,nitkjd_profile_code,nitkjd_licence
0,47010,1,0.0,21.0,Ap,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47010,2,21.0,35.0,E1,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Extract soil properties for all samples

In [5]:
def calc_mean_val_in_depth(df, profile_id, val_name, depth_thred_upper, depth_thred_lower):
    '''Calculate the mean value of a certain soil property between the upper depth and lower depth in a soil profile
    '''
    df_one_profile = df[df['profile_id'] == profile_id].reset_index(drop=True)
    val_mean = 0.0
    depth_length_sum = 0.0
    for i in range(len(df_one_profile)):
        if df_one_profile['lower_depth'][i] >= depth_thred_upper and df_one_profile['upper_depth'][i] <= depth_thred_lower:
            depth_length = min(depth_thred_lower, df_one_profile['lower_depth'][i]) - max(depth_thred_upper, df_one_profile['upper_depth'][i])
            value = df_one_profile[val_name][i]
            if np.isnan(value):
                continue
            val_mean += df_one_profile[val_name][i] * depth_length
            depth_length_sum += depth_length
    if depth_length_sum <= 0:
        val_mean = -1
    # elif len(df_one_profile) <= 1:
    #     val_mean = -1
    else:
        val_mean = val_mean / depth_length_sum

    # calculate based on spline function
    # spl = interpolate.splrep(x=depth_list, y=value_list, k=2)
    # depth_spl_list = np.linspace(0, 100, 100)
    # value_spl_list = interpolate.splev(depth_spl_list, spl)

    return val_mean
    
# profile_id = 36898
# soc_mean = calc_mean_val_in_depth(df=df_soc, profile_id=profile_id, val_name='SOC', depth_thred_upper=0, depth_thred_lower=30)
# print(soc_mean)

In [6]:
print([featname for featname in list(df_physical.columns) if '_value_avg' in featname])
print()
print([featname for featname in list(df_chemical.columns) if '_value_avg' in featname])

['bdfi33_value_avg', 'bdfiad_value_avg', 'bdfifm_value_avg', 'bdfiod_value_avg', 'bdws33_value_avg', 'bdwsad_value_avg', 'bdwsfm_value_avg', 'bdwsod_value_avg', 'clay_value_avg', 'cfgr_value_avg', 'cfvo_value_avg', 'sand_value_avg', 'silt_value_avg', 'wg0100_value_avg', 'wg0010_value_avg', 'wg1500_value_avg', 'wg0200_value_avg', 'wg0033_value_avg', 'wg0500_value_avg', 'wg0006_value_avg', 'wv0100_value_avg', 'wv0010_value_avg', 'wv1500_value_avg', 'wv0200_value_avg', 'wv0033_value_avg', 'wv0500_value_avg', 'wv0006_value_avg']

['tceq_value_avg', 'cecph7_value_avg', 'cecph8_value_avg', 'ecec_value_avg', 'elco20_value_avg', 'elco25_value_avg', 'elco50_value_avg', 'elcosp_value_avg', 'orgc_value_avg', 'phca_value_avg', 'phaq_value_avg', 'phkc_value_avg', 'phnf_value_avg', 'phpbyi_value_avg', 'phpmh3_value_avg', 'phpols_value_avg', 'phprtn_value_avg', 'phptot_value_avg', 'phpwsl_value_avg', 'totc_value_avg', 'nitkjd_value_avg']


In [ ]:
val_name_list = ['clay', 'sand', 'silt']

df_res = pd.DataFrame()
df_res['profile_id'] = profile_id_all_list

for val_name in val_name_list:
    print('Processing <{}>...'.format(val_name))
    val_0to30_list = []
    val_30to100_list = []
    for i in tqdm(range(len(profile_id_all_list))):
        profile_id = profile_id_all_list[i]
        val_0to30 = calc_mean_val_in_depth(df=df_physical, profile_id=profile_id, val_name=val_name+'_value_avg', depth_thred_upper=0, depth_thred_lower=30)
        val_30to100 = calc_mean_val_in_depth(df=df_physical, profile_id=profile_id, val_name=val_name+'_value_avg', depth_thred_upper=30, depth_thred_lower=100)

        val_0to30_list.append(val_0to30)
        val_30to100_list.append(val_30to100)

    df_res[val_name+'_0to30'] = val_0to30_list
    df_res[val_name+'_30to100'] = val_30to100_list
    print('Valid data proportion <{}>:\t {:.1f}%'.format(val_name+'_0to30', 100*len(df_res[df_res[val_name+'_0to30'] != -1]) / len(df_res)))
    print('Valid data proportion <{}>:\t {:.1f}%'.format(val_name+'_30to100', 100*len(df_res[df_res[val_name+'_30to100'] != -1]) / len(df_res)))
    print()
print(df_res.shape)
df_res.head()

In [ ]:
val_name_list = ['cecph7', 'nitkjd', 'phaq']

# df_res = pd.DataFrame()
# df_res['profile_id'] = profile_id_all_list

for val_name in val_name_list:
    print('Processing <{}>...'.format(val_name))
    val_0to30_list = []
    val_30to100_list = []
    for i in tqdm(range(len(profile_id_all_list))):
        profile_id = profile_id_all_list[i]
        val_0to30 = calc_mean_val_in_depth(df=df_chemical, profile_id=profile_id, val_name=val_name+'_value_avg', depth_thred_upper=0, depth_thred_lower=30)
        val_30to100 = calc_mean_val_in_depth(df=df_chemical, profile_id=profile_id, val_name=val_name+'_value_avg', depth_thred_upper=30, depth_thred_lower=100)

        val_0to30_list.append(val_0to30)
        val_30to100_list.append(val_30to100)

    df_res[val_name+'_0to30'] = val_0to30_list
    df_res[val_name+'_30to100'] = val_30to100_list
    print('Valid data proportion <{}>:\t {:.1f}%'.format(val_name+'_0to30', 100*len(df_res[df_res[val_name+'_0to30'] != -1]) / len(df_res)))
    print('Valid data proportion <{}>:\t {:.1f}%'.format(val_name+'_30to100', 100*len(df_res[df_res[val_name+'_30to100'] != -1]) / len(df_res)))
    print()
print(df_res.shape)
df_res.head()

In [ ]:
# df_res.to_csv('../datasets/covariates/wosis/samples_soil_properties.csv', index=False)